# HTMDEC DMS API Example

This example demonstrates how to use the new HTMDEC DMS `/form` and `/entry` REST API endpoints to search for BIRDSHOT data which can be converted to a Pandas dataframe for analysis and visualization. 

The `birdshot.py` module implements some example utility functions for querying, analyzing, and visualizing BIRDSHOT data.

### Note:
During the ingest process, form data collected by Contextualize in the Campaign 1 structure was converted to the [Campaign 2 structure](https://docs.google.com/document/d/1FpiXwLQi8QAOuLB0Xr80V14qGfqtyCpT/edit). This means that:
* Sample IDs were converted to the Campaign 2 structure 
* Campaign 1 "child" sample numbers (T*nn*) have been replaced with ordinal letters (a, b, c, ...)


### Using the REST API

The [Girder Client](https://girder.readthedocs.io/en/latest/python-client.html) can be used to query the REST endpoints directly. When running via the DMS, the API URL and user token required for access are available in the current environment.


In [1]:
from girder_client import GirderClient
import os

client = GirderClient(apiUrl=os.environ['GIRDER_API_URL'])
client.token = os.environ['GIRDER_TOKEN']

To query VAM data from iteration 1 (AAA) across all forms, use the `/entry/search` endpoint to query the Sample ID. Data is returned in JSON:

In [2]:
iteration = 'BBB'
raw_data = client.get(
        'entry/search', parameters={'query': f'^{iteration}.._VAM-.', 'limit': 1000}
)
raw_data[0]["data"]

IndexError: list index out of range

Use the `/form` and `entry` endpoints to query data from a specific form:

In [3]:
form_name = 'tensile-details.json'
form = client.get('form', parameters={'entryFileName': form_name, 'limit': 1000})
form[0]['_id']

tensile_data = client.get('entry', parameters={'formId': form[0]['_id'], 'limit': 1000})
tensile_data[0]["data"]

IndexError: list index out of range

### Using the `birdshot` Module

The `birdshot.py` module implements a few helper functions to query the REST API and convert data into a single Pandas dataframe for analysis:

In [4]:
import birdshot

SyntaxError: 'return' outside function (birdshot.py, line 78)

The `query()` method takes the iteration identifier as an argument and returns a dataframe of results from multiple characterization methods. The example dataframe is intended to reproduce the information available in the Summary Synthesis Results (for example see [HTMDEC AAB Summary Synthesis Results](https://docs.google.com/spreadsheets/d/15cdImpOComsvUpAIq20_nyff65WVzN_q/)).

In [5]:
df = birdshot.query("BBB")
df

2024-09-27 16:47:52,634 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): girder:8080
2024-09-27 16:47:52,648 - urllib3.connectionpool - DEBUG - http://girder:8080 "GET /api/v1/entry/search?query=%5EBBB.._VAM-.&limit=1000 HTTP/1.1" 200 122187


,Target Composition (%).Al,Target Composition (%).Co,Target Composition (%).Cr,Target Composition (%).Cu,Target Composition (%).Fe,Target Composition (%).Mn,Target Composition (%).Ni,Target Composition (%).V,Measured Composition (%).Al,Measured Composition (%).Co,...,Maximum ∂2σ/∂ε2.b,UTS/YS Ratio.b,Ultimate Tensile Strength.b,Yield Strength.b,Elastic Modulus.c,Elongation.c,Maximum ∂2σ/∂ε2.c,UTS/YS Ratio.c,Ultimate Tensile Strength.c,Yield Strength.c
BBB01_VAM-A,0,24,4,0,8,4,36,24,0.000000,20.500000,...,189.089549,2.043199,900.951997,440.951709,NaN,NaN,NaN,NaN,NaN,NaN
BBB02_VAM-A,4,32,8,4,4,20,24,4,4.229333,30.367333,...,682.271027,2.250473,693.737195,308.262832,NaN,NaN,NaN,NaN,NaN,NaN
BBB03_VAM-A,4,0,12,4,4,16,52,8,4.076667,0.000000,...,923.053457,2.300015,700.587065,304.601042,NaN,NaN,NaN,NaN,NaN,NaN
BBB04_VAM-A,4,12,4,0,28,0,40,12,3.987333,11.112667,...,1073.308285,2.528193,657.229928,259.960380,195.301417,1.0,1421.265508,2.539381,670.187763,263.917801
BBB05_VAM-A,0,44,12,0,4,4,12,24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BBB06_VAM-A,0,16,20,0,4,4,36,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BBB07_VAM-A,4,40,0,0,12,4,36,4,3.987333,39.097333,...,369.055902,2.754701,572.582187,207.856377,NaN,NaN,NaN,NaN,NaN,NaN
BBB08_VAM-A,4,8,12,4,4,24,40,4,3.844667,8.033333,...,1692.804326,2.388512,670.565536,280.746119,162.079509,1.0,1196.533894,2.333373,662.756447,284.033656
BBB09_VAM-A,0,4,8,0,4,28,40,16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BBB10_VAM-A,4,0,4,4,8,16,52,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
